In [5]:
def inference(x):
    init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", [784, 10], initializer=init)
    b = tf.get_variable("b", [10], initializer=init)
    output = tf.nn.softmax(tf.matmul(x,W) + b)
    return output

def loss(output, y):
    dot_product = y * tf.log(output)
    xentropy = -tf.reduce_sum(dot_product, reduction_indices=1)
    loss = tf.reduce_mean(xentropy)
    
    return loss

def training(cost, global_step):
    optimizer = tf.train.GradientDescentoptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

def evaluate(output,y):
    correct_prediction = tf.equal(tf.argmax(output,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [6]:
# 각 미니배치에 대한 비용과 검증 오차, 파라미터들의 분포를 기록하는 명령
# tf.summary.scalar19 와 tf.summary.histogram을 사용한다.

def training(cost, global_step):
    tf.summary.scalar("cost",cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op


In [11]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# 파라미터

learning_rate = 0.01
training_epochs = 100
batch_size = 100
display_step = 1

with tf.Graph().as_default():
    # 28*28=784 MNIST 데이터 이미지
    x = tf.placeholder("float", [None,784])
    
    # 0-9숫자인식 = > 10 클래스
    y = tf.placeholder("float", [None,10])
    
    output = inference(x)
    
    cost = loss(output, y)
    
    global_step = tf.Variable(0, name='global_step', trainable=False)
    
    train_op = training(cost, global_step)
    
    eval_op = evaluate(output, y)
    
    summary_op = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    
    sess = tf.Session()
    
    summary_writer = tf.summary.FileWriter("logistic_logs/", graph_def=sess.graph_def)
    
    init_op = tf.initialize_all_variables()
    
    sess.run(init_op)
    
    # 학습주기
    
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            mbatch_x, mbatch_y = mnist.train.next_batch(batch_size)
            
            feed_dict = {x : mbatch_x, y:mbatch_y}
            sess.run(train_op, feed_dict = feed_dict)
            
            minibatch_cost = sess.run(cost, feed_dict = feed_dict)
            avg_cost += minibatch_cost/total_batch
            
        # epoch마다 기록 표시
        
        if epoch % display_step == 0:
            val_feed_dict = {
                x : mnist.validation.images,
                y : mnist.validation.labels
            }
            
            accuracy = sess.run(eval_op, feed_dict=val_feed_dict)
            
            print("Validation Error:", (1-accuracy))
            
            summary_str = sess.run(summary_op, feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, sess.run(global_step))
            
            saver.save(sess, "logistic_logs/model-checkpoint", 
                      global_step = global_step)
            
        print("Optimization Finished!")
        
        test_feed_dict = {
            x : mnist.test.images,
            y : mnist.test.labels
        }
        
        accuracy = sess.run(eval_op, feed_dict=test_feed_dict)
        
        print("Test Accuracy : ", accuracy)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Validation Error: 0.147800028324
Optimization Finished!
Test Accuracy :  0.855
Validation Error: 0.129000008106
Optimization Finished!
Test Accuracy :  0.8732
Validation Error: 0.120999991894
Optimization Finished!
Test Accuracy :  0.8821
Validation Error: 0.114600002766
Optimization Finished!
Test Accuracy :  0.8867
Validation Error: 0.110000014305
Optimization Finished!
Test Accuracy :  0.8922
Validation Error: 0.106800019741
Optimization Finished!
Test Accuracy :  0.8947
Validation Error: 0.105000019073
Optimization Finished!
Test Accuracy :  0.8984
Validation Error: 0.101800024509
Optimization Finished!
Test Accuracy :  0.8998
Validation Error: 0.099799990654
Optimization Finished!
Test Accuracy :  0.901
Validation Error: 0.100199997425
Optimization Finished!
Test Accuracy :  0.9024
Validati

Validation Error: 0.0753999948502
Optimization Finished!
Test Accuracy :  0.922
Validation Error: 0.0749999880791
Optimization Finished!
Test Accuracy :  0.9221
